Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Anurag Pathak"
COLLABORATORS = ""

---

In [120]:
##Importing the dataset and cleaning it
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
df = pd.read_csv("ElectricVehicle.csv")
null_counts = df.isnull().sum()
#print(null_counts)

#things to consider when cleaning data:
#Understanding data types
#Null values
#Duplicates
#Outliers
#Inconsistent Values
#One-hot Encoding for categorical variables
#Correct Data types
#imputing values

#Cleaning Null Values (Null Values and Inconsistent Values) and imputing values
#When looking at the variables who have null value for County, City, and Postal Code
#we see 5 entries, who seem to missing a lot of values. For that reason, we are removing those
#entries entirely due to their being a lack of information to attempt to try and impute a value
df = df[df["County"].isnull() == False]
#replacing nan with 0's
#for the Base MSRP and Electric Range variables, I plan on replacing the NAN variables with 0's
#I'm doing this so that its easier to replace the 0 values under the Electric Range variable, since
# a vehicle with a 0 electric range isn't actually a EV. There could be a lot of reasons why its 0,
#but the main one is likely lack of proper data collection. For that reason, we are removing any 0 value.
count_zero_range = (df["Electric Range"] == 0).sum()
#print(count_zero_range)
df['Base MSRP'] = df['Base MSRP'].fillna(0)
df['Electric Range'] = df['Electric Range'].fillna(0)
df['Legislative District'] = df['Legislative District'].fillna("Unknown")
df['Vehicle Location'] = df['Vehicle Location'].fillna("")
df = df[df['Electric Range']!= 0]
df = df[df['Base MSRP']!= 0]
#print(len(df))
null_counts = df.isnull().sum()
#print(null_counts)
#Since the only nan values left are for legistlative District and vehicle location, which is a variable i won't be interacting with,
#this part of the cleaning process is complete. 

#Duplicates
#Luckily for us, this dataset only uses one entry-per car, meaning that we wouldn't need to check for duplicates. That being said, there is no way for us
#to 100% make sure this is the case, so we just need to trust what they are saying since the vin variable only includes the first 10 characters, which 
#isn't enough to identify a duplicate or not.

#One-hot encoding
columns_to_encode = ['Model Year', 'Make', 'Model', 'Electric Vehicle Type']
df = pd.get_dummies(df, columns=columns_to_encode)
#print(df)

#unique_makes = df['Make'].unique()
#print(unique_makes)
#unique_models = df['Model'].unique()
#print(unique_models)
#unique_model_year = df['Model Year'].unique()
#print(unique_model_year)
#unique_electric_range = df['Electric Range'].unique()
#print(unique_electric_range)
#unique_EVType = df['Electric Vehicle Type'].unique()
#print(unique_EVType)
# columns_to_check = ['Make', 'Model Year', 'Model', 'Electric Range']
# for col in columns_to_check:
#     null_count = df[col].isnull().sum()
#     print(f"{col} has {null_count} empty values.")

#outliers
#I am using the interquartile method to get rid of outliers. 
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1                 
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
df = remove_outliers_iqr(df, 'Electric Range')
df = remove_outliers_iqr(df, 'Base MSRP')


null_counts = df.isnull().sum()
print(null_counts)
print(len(df))
df.to_csv('cleanedDF.csv', index=False)


VIN (1-10)                                                      0
County                                                          0
City                                                            0
State                                                           0
Postal Code                                                     0
Clean Alternative Fuel Vehicle (CAFV) Eligibility               0
Electric Range                                                  0
Base MSRP                                                       0
Legislative District                                            0
DOL Vehicle ID                                                  0
Vehicle Location                                                0
Electric Utility                                                0
2020 Census Tract                                               0
Model Year_2008                                                 0
Model Year_2010                                                 0
Model Year

In [ ]:
#Exploratory Data Analysis
import seaborn as sns
import matplotlib.pyplot as plt

correlation_columns = ['Electric Range', 'Base MSRP'] + [col for col in df.columns if df[col].dtype in ['int64', 'float64']]
correlation_data = df[correlation_columns].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_data, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()

# Calculate averages for BEV and PHEV
bev_avg = df[df["Electric Vehicle Type_Battery Electric Vehicle (BEV)"] == 1][["Electric Range", "Base MSRP"]].mean()
phev_avg = df[df["Electric Vehicle Type_Plug-in Hybrid Electric Vehicle (PHEV)"] == 1][["Electric Range", "Base MSRP"]].mean()
print(f"BEV Averages:\n{bev_avg}")
print(f"PHEV Averages:\n{phev_avg}")

#Adding a new column with actual value - average
df['Abs Diff Electric Range'] = df.apply(
    lambda row: abs(row['Electric Range'] - (bev_avg['Electric Range'] if row["Electric Vehicle Type_Battery Electric Vehicle (BEV)"] == 1 else phev_avg['Electric Range'])), 
    axis=1
)
df['Abs Diff Base MSRP'] = df.apply(
    lambda row: abs(row['Base MSRP'] - (bev_avg['Base MSRP'] if row["Electric Vehicle Type_Battery Electric Vehicle (BEV)"] == 1 else phev_avg['Base MSRP'])), 
    axis=1
)
print(df[['Abs Diff Electric Range', 'Abs Diff Base MSRP']].head())

plt.figure(figsize=(8, 6))
sns.scatterplot(data=df, x='Electric Range', y='Base MSRP', hue='Electric Vehicle Type_Battery Electric Vehicle (BEV)')
plt.title("Electric Range vs. Base MSRP")
plt.xlabel("Electric Range")
plt.ylabel("Base MSRP")
plt.legend(title="Vehicle Type", labels=["PHEV", "BEV"])
plt.show()

plt.figure(figsize=(8, 6))
sns.histplot(data=df, x='Electric Range', hue='Electric Vehicle Type_Battery Electric Vehicle (BEV)', kde=True, bins=30)
plt.title("Distribution of Electric Range by Vehicle Type")
plt.xlabel("Electric Range")
plt.show()

averages = pd.DataFrame({'Type': ['BEV', 'PHEV'], 
                         'Avg Electric Range': [bev_avg['Electric Range'], phev_avg['Electric Range']],
                         'Avg Base MSRP': [bev_avg['Base MSRP'], phev_avg['Base MSRP']]})

averages.set_index('Type').plot(kind='bar', figsize=(8, 6), title="Average Electric Range and Base MSRP by Vehicle Type")
plt.ylabel("Value")
plt.show()


In [104]:
#SQL Component
import sqlite3

conn = sqlite3.connect("electric_vehicles.db")
df.to_sql("vehicles", conn, if_exists="replace", index=False)

create_bev_table = """
CREATE TABLE IF NOT EXISTS bev AS
SELECT * FROM vehicles
WHERE "Electric Vehicle Type_Battery Electric Vehicle (BEV)" = 1;
"""
create_phev_table = """
CREATE TABLE IF NOT EXISTS phev AS
SELECT * FROM vehicles
WHERE "Electric Vehicle Type_Plug-in Hybrid Electric Vehicle (PHEV)" = 1;
"""


cursor = conn.cursor()
cursor.execute(create_bev_table)
cursor.execute(create_phev_table)
conn.commit()

print("Separate tables 'bev' and 'phev' created.")



query_model_make_bev = """
SELECT 'BMW' AS Make, AVG("Base MSRP") AS Avg_Base_MSRP, AVG("Electric Range") AS Avg_Electric_Range
FROM bev WHERE "Make_BMW" = 1
UNION ALL
SELECT 'CADILLAC', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Make_CADILLAC" = 1
UNION ALL
SELECT 'CHRYSLER', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Make_CHRYSLER" = 1
UNION ALL
SELECT 'FISKER', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Make_FISKER" = 1
UNION ALL
SELECT 'KIA', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Make_KIA" = 1
UNION ALL
SELECT 'MINI', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Make_MINI" = 1
UNION ALL
SELECT 'PORSCHE', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Make_PORSCHE" = 1
UNION ALL
SELECT 'SUBARU', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Make_SUBARU" = 1
UNION ALL
SELECT 'TESLA', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Make_TESLA" = 1
UNION ALL
SELECT 'VOLVO', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Make_VOLVO" = 1
UNION ALL
SELECT 'WHEEGO ELECTRIC CARS', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Make_WHEEGO ELECTRIC CARS" = 1
"""

query_model_make_phev = """
SELECT 'BMW' AS Make, AVG("Base MSRP") AS Avg_Base_MSRP, AVG("Electric Range") AS Avg_Electric_Range
FROM phev WHERE "Make_BMW" = 1 
UNION ALL
SELECT 'CADILLAC', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Make_CADILLAC" = 1
UNION ALL
SELECT 'CHRYSLER', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Make_CHRYSLER" = 1
UNION ALL
SELECT 'FISKER', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Make_FISKER" = 1
UNION ALL
SELECT 'KIA', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Make_KIA" = 1
UNION ALL
SELECT 'MINI', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Make_MINI" = 1
UNION ALL
SELECT 'PORSCHE', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Make_PORSCHE" = 1
UNION ALL
SELECT 'SUBARU', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Make_SUBARU" = 1
UNION ALL
SELECT 'TESLA', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Make_TESLA" = 1
UNION ALL
SELECT 'VOLVO', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Make_VOLVO" = 1
UNION ALL
SELECT 'WHEEGO ELECTRIC CARS', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Make_WHEEGO ELECTRIC CARS" = 1
"""


query_model_year_bev = """
SELECT '2008' AS Model_Year, AVG("Base MSRP") AS Avg_Base_MSRP, AVG("Electric Range") AS Avg_Electric_Range
FROM bev WHERE "Model Year_2008" = 1
UNION ALL
SELECT '2010', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Model Year_2010" = 1
UNION ALL
SELECT '2011', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Model Year_2011" = 1
UNION ALL
SELECT '2012', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Model Year_2012" = 1
UNION ALL
SELECT '2013', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Model Year_2013" = 1
UNION ALL
SELECT '2014', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Model Year_2014" = 1
UNION ALL
SELECT '2015', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Model Year_2015" = 1
UNION ALL
SELECT '2016', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Model Year_2016" = 1
UNION ALL
SELECT '2017', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Model Year_2017" = 1
UNION ALL
SELECT '2018', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Model Year_2018" = 1
UNION ALL
SELECT '2019', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Model Year_2019" = 1
UNION ALL
SELECT '2020', AVG("Base MSRP"), AVG("Electric Range")
FROM bev WHERE "Model Year_2020" = 1
"""

query_model_year_phev = """
SELECT '2008' AS Model_Year, AVG("Base MSRP") AS Avg_Base_MSRP, AVG("Electric Range") AS Avg_Electric_Range
FROM phev WHERE "Model Year_2008" = 1
UNION ALL
SELECT '2010', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Model Year_2010" = 1
UNION ALL
SELECT '2011', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Model Year_2011" = 1
UNION ALL
SELECT '2012', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Model Year_2012" = 1
UNION ALL
SELECT '2013', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Model Year_2013" = 1
UNION ALL
SELECT '2014', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Model Year_2014" = 1
UNION ALL
SELECT '2015', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Model Year_2015" = 1
UNION ALL
SELECT '2016', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Model Year_2016" = 1
UNION ALL
SELECT '2017', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Model Year_2017" = 1
UNION ALL
SELECT '2018', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Model Year_2018" = 1
UNION ALL
SELECT '2019', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Model Year_2019" = 1
UNION ALL
SELECT '2020', AVG("Base MSRP"), AVG("Electric Range")
FROM phev WHERE "Model Year_2020" = 1
"""

# Fetch data
bev_model_year_df = pd.read_sql_query(query_model_year_bev, conn)
phev_model_year_df = pd.read_sql_query(query_model_year_phev, conn)
bev_model_year_df["Vehicle_Type"] = "BEV"
phev_model_year_df["Vehicle_Type"] = "PHEV"
combined_model_year_df = pd.concat([bev_model_year_df, phev_model_year_df])
print(combined_model_year_df)
print()

bev_make_df = pd.read_sql_query(query_model_make_bev, conn)
phev_make_df = pd.read_sql_query(query_model_make_phev, conn)
bev_make_df["Vehicle_Type"] = "BEV"
phev_make_df["Vehicle_Type"] = "PHEV"
combined_make_df = pd.concat([bev_make_df, phev_make_df])
print(combined_make_df)


conn.close()


Separate tables 'bev' and 'phev' created.
   Model_Year  Avg_Base_MSRP  Avg_Electric_Range Vehicle_Type
0        2008   98950.000000          220.000000          BEV
1        2010  110950.000000          245.000000          BEV
2        2011  109000.000000          245.000000          BEV
3        2012   59900.000000          265.000000          BEV
4        2013   69900.000000          208.000000          BEV
5        2014   69900.000000          208.000000          BEV
6        2015            NaN                 NaN          BEV
7        2016   31950.000000           93.000000          BEV
8        2017   32250.000000           93.000000          BEV
9        2018   33950.000000          111.000000          BEV
10       2019            NaN                 NaN          BEV
11       2020            NaN                 NaN          BEV
0        2008            NaN                 NaN         PHEV
1        2010   32995.000000          100.000000         PHEV
2        2011            NaN

In [123]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

#df = pd.get_dummies(df, columns=["Make", "Electric Vehicle Type"], drop_first=True)

scaler = MinMaxScaler()
df[['Electric Range']] = scaler.fit_transform(df[['Electric Range']])
target_scaler = MinMaxScaler()
df['Base MSRP'] = target_scaler.fit_transform(df[['Base MSRP']])

features_model_1 = ["Electric Range"] + [col for col in df.columns if col.startswith("Make_")] + [col for col in df.columns if col.startswith("Model Year_")]
features_model_2 = features_model_1 + [col for col in df.columns if col.startswith("Electric Vehicle Type_")]
target = "Base MSRP"

# Split data
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(df[features_model_1], df[target], test_size=0.15, random_state=42)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(df[features_model_2], df[target], test_size=0.15, random_state=42)

# Fit and evaluate model 1
model_1 = LinearRegression()
model_1.fit(X_train_1, y_train_1)
y_pred_1 = model_1.predict(X_test_1)
mse_1 = mean_squared_error(y_test_1, y_pred_1)
r2_1 = r2_score(y_test_1, y_pred_1)
print("Model 1 Results")
print(f"Mean Squared Error: {mse_1:.5f}")
print(f"R2 Score: {r2_1:.5f}")

# Fit and evaluate model 2
model_2 = LinearRegression()
model_2.fit(X_train_2, y_train_2)
y_pred_2 = model_2.predict(X_test_2)
mse_2 = mean_squared_error(y_test_2, y_pred_2)
r2_2 = r2_score(y_test_2, y_pred_2)
print("\nModel 2 Results")
print(f"Mean Squared Error: {mse_2:.5f}")
print(f"R2 Score: {r2_2:.5f}")

print("Model 1 Coefficients:")
for feature, coef in zip(features_model_1, model_1.coef_):
    print(f"{feature}: {coef:.5f}")
print("\nModel 2 Coefficients:")
for feature, coef in zip(features_model_2, model_2.coef_):
    print(f"{feature}: {coef:.5f}")


Model 1 Results
Mean Squared Error: 0.00281
R2 Score: 0.93934

Model 2 Results
Mean Squared Error: 0.00281
R2 Score: 0.93935
Model 1 Coefficients:
Electric Range: -0.65947
Make_BMW: -126976169758.60660
Make_CADILLAC: -126976169758.28261
Make_CHRYSLER: -126976169758.73779
Make_FISKER: -43644966435.42239
Make_KIA: -126976169758.60306
Make_MINI: -126976169758.83083
Make_PORSCHE: -1703658602796.05908
Make_SUBARU: -126976169758.83871
Make_TESLA: -43644966435.35330
Make_VOLVO: -126976169758.57185
Make_WHEEGO ELECTRIC CARS: -43644966436.71738
Model Year_2008: -40733654422.22498
Model Year_2010: -40733654422.00822
Model Year_2011: -40733654422.03277
Model Year_2012: -40733654422.60309
Model Year_2013: -40733654422.62437
Model Year_2014: -40733654422.62312
Model Year_2015: -89988873.56567
Model Year_2016: 42597548899.84392
Model Year_2017: 42597548899.84509
Model Year_2018: 42597548899.93079
Model Year_2019: 42597548899.92371
Model Year_2020: 1619279981937.71460

Model 2 Coefficients:
Electric 

---